In [6]:
import os
from pathlib import Path
from freqtrade.configuration import Configuration
from freqtrade.data.dataprovider import DataProvider
from freqtrade.resolvers import StrategyResolver

project_root = "/Users/yutieyang/Documents/yuty/yuty_projects/money_game/Trading/"
config_path = "/Users/yutieyang/Documents/yuty/yuty_projects/money_game/Trading/user_data/configs/config_test_futrue_IntradayMomentum.json"
data_path = "/Users/yutieyang/Documents/yuty/yuty_projects/money_game/Data/binance/Vol_top20_futrue_20250101_20250411"
strategy = "IntradayMomentum"

os.chdir(project_root)
config = Configuration.from_files([config_path])

# Define some constants
config["timeframe"] = "5m"
# Name of the strategy class
config["strategy"] = "IntradayMomentum"
# Pair to analyze - Only use one pair here
pair = "BTC/USDT:USDT"


2025-04-13 20:12:41,075 - freqtrade.configuration.load_config - INFO - Using config: 
/Users/yutieyang/Documents/yuty/yuty_projects/money_game/Trading/user_data/configs/config_test_futrue_IntradayMomentum.json ...

2025-04-13 20:12:41,080 - freqtrade.loggers - INFO - Enabling colorized output.

2025-04-13 20:12:41,081 - root - INFO - Logfile configured

2025-04-13 20:12:41,082 - freqtrade.loggers - INFO - Verbosity set to 0

2025-04-13 20:12:41,084 - freqtrade.configuration.configuration - INFO - Using user-data directory: /Users/yutieyang/Documents/yuty/yuty_projects/money_game/Trading/user_data ...

2025-04-13 20:12:41,085 - freqtrade.configuration.configuration - INFO - Using data directory: /Users/yutieyang/Documents/yuty/yuty_projects/money_game/Trading/user_data/data/binance ...

2025-04-13 20:12:41,087 - freqtrade.exchange.check_exchange - INFO - Checking exchange...

2025-04-13 20:12:41,099 - freqtrade.exchange.check_exchange - INFO - Exchange "binance" is officially supported by the Freqtrade development team.

2025-04-13 20:12:41,100 - freqtrade.configuration.configuration - INFO - Using pairlist from configuration.

# Add Key Variables

In [22]:
# Load data using values set above
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType


candles = load_pair_history(
    datadir=Path(data_path),
    timeframe=config["timeframe"],
    pair=pair,
    data_format="feather",  # Make sure to update this to your data
    candle_type=CandleType.FUTURES,
)

# Confirm success
print(f"Loaded {len(candles)} rows of data for {pair} from {data_path}")
candles.head()


Loaded 29214 rows of data for BTC/USDT:USDT from /Users/yutieyang/Documents/yuty/yuty_projects/money_game/Data/binance/Vol_top20_futrue_20250101_20250411


,date,open,high,low,close,volume
0,2025-01-01 00:00:00+00:00,93548.8,93690.0,93514.2,93648.4,341.604
1,2025-01-01 00:05:00+00:00,93648.3,93666.7,93576.9,93585.9,166.341
2,2025-01-01 00:10:00+00:00,93586.0,93637.2,93460.2,93631.2,480.204
3,2025-01-01 00:15:00+00:00,93631.2,93829.4,93588.0,93781.0,545.647
4,2025-01-01 00:20:00+00:00,93781.0,93885.0,93680.0,93712.0,392.042


In [ ]:
import pandas as pd
import numpy as np


In [27]:
dataframe = candles

# Calculate VWAP
hlc = (dataframe['high'] + dataframe['low'] + dataframe['close']) / 3
dataframe['vwap'] = (hlc * dataframe['volume']).cumsum() / dataframe['volume'].cumsum()

# Calculate rolling mean and sigma
dataframe['move_open'] = (dataframe['close'] / dataframe['open'] - 1).abs()
dataframe['move_open_rolling_mean'] = dataframe['move_open'].rolling(window=14, min_periods=13).mean()
dataframe['sigma_open'] = dataframe['move_open_rolling_mean'].shift(1)


In [29]:
dataframe.tail()


,open,high,low,close,volume,day,move_open,vwap,spy_dvol,min_from_open,minute_of_day,move_open_rolling_mean,sigma_open
caldt,,,,,,,,,,,,,
2025-04-12 10:05:00+00:00,83473.6,83533.1,83419.0,83525.4,165.532,2025-04-12,0.000621,90582.345905,0.035504,36.0,36,0.000805,0.000795
2025-04-12 10:10:00+00:00,83525.4,83533.3,83444.0,83452.3,106.052,2025-04-12,0.000875,90582.312619,0.035504,41.0,41,0.000755,0.000805
2025-04-12 10:15:00+00:00,83452.3,83469.9,83355.5,83363.2,237.412,2025-04-12,0.001068,90582.237262,0.035504,46.0,46,0.000779,0.000755
2025-04-12 10:20:00+00:00,83363.2,83428.1,83346.2,83421.9,307.204,2025-04-12,0.000704,90582.139789,0.035504,51.0,51,0.000665,0.000779
2025-04-12 10:25:00+00:00,83421.9,83481.6,83421.8,83441.6,144.664,2025-04-12,0.000236,90582.094207,0.035504,56.0,56,0.000660,0.000665


## Load and run strategy
* Rerun each time the strategy file is changed

In [5]:
# Load strategy using values set above



strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)
strategy.ft_bot_start()

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {"pair": pair})
df.tail()


2025-04-13 20:04:30,553 - freqtrade.resolvers.iresolver - INFO - Using resolved strategy IntradayMomentum from 
'/Users/yutieyang/Documents/yuty/yuty_projects/money_game/Trading/user_data/strategies/IntradayMomentum.py'...

2025-04-13 20:04:30,554 - freqtrade.strategy.hyper - INFO - Found no parameter file.

2025-04-13 20:04:30,555 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'timeframe' with value in config file: 8h.

2025-04-13 20:04:30,556 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'stake_currency' with value in config file: USDT.

2025-04-13 20:04:30,557 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'stake_amount' with value in config file: unlimited.

...

2025-04-13 20:04:30,579 - freqtrade.resolvers.strategy_resolver - INFO - Strategy using max_open_trades: 5

2025-04-13 20:04:30,580 - freqtrade.strategy.hyper - INFO - No params for buy found, using default values.

2025-04-13 20:04:30,592 - freqtrade.strategy.hyper - INFO - No params for sell found, using default values.

2025-04-13 20:04:30,594 - freqtrade.strategy.hyper - INFO - No params for protection found, using default values.

,date,open,high,low,close,volume,obv,trend,enter_tag,enter_long,enter_short,exit_tag,exit_long,exit_short
298,2025-04-10 08:00:00+00:00,81561.5,82488.0,78559.0,78731.7,143121.765,-2363101.889,80275.410148,,NaN,1.0,,NaN,NaN
299,2025-04-10 16:00:00+00:00,78731.7,80011.8,78416.0,79560.2,87817.839,-2275284.050,80207.294896,,NaN,NaN,,NaN,NaN
300,2025-04-11 00:00:00+00:00,79560.3,81388.0,78935.5,81213.6,71931.272,-2203352.778,80303.133477,,1.0,NaN,,NaN,NaN
301,2025-04-11 08:00:00+00:00,81213.7,83297.9,80691.3,82486.7,148681.007,-2054671.771,80511.092194,,NaN,NaN,,NaN,NaN
302,2025-04-11 16:00:00+00:00,82486.8,84300.0,81968.0,83378.3,75165.592,-1979506.179,80784.159604,,NaN,NaN,,NaN,NaN


### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


In [ ]:
# Report results
print(f"Generated {df['enter_long'].sum()} entry signals")
data = df.set_index("date", drop=False)
data.tail()


## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [ ]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats


# if backtest_dir points to a directory, it'll automatically load the last backtest file.
backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir can also point to a specific file
# backtest_dir = (
#   config["user_data_dir"] / "backtest_results/backtest-result-2020-07-01_20-04-22.json"
# )


In [ ]:
# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

strategy = "SampleStrategy"
# All statistics are available per strategy, so if `--strategy-list` was used during backtest,
# this will be reflected here as well.
# Example usages:
print(stats["strategy"][strategy]["results_per_pair"])
# Get pairlist used for this backtest
print(stats["strategy"][strategy]["pairlist"])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats["strategy"][strategy]["market_change"])
# Maximum drawdown ()
print(stats["strategy"][strategy]["max_drawdown_abs"])
# Maximum drawdown start and end
print(stats["strategy"][strategy]["drawdown_start"])
print(stats["strategy"][strategy]["drawdown_end"])


# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats["strategy_comparison"])


In [ ]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["exit_reason"].value_counts()


## Plotting daily profit / equity line

In [ ]:
# Plotting equity line (starting with 0 on day 1 and adding daily profit for each backtested day)

import pandas as pd
import plotly.express as px

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_stats


# strategy = 'SampleStrategy'
# config = Configuration.from_files(["user_data/config.json"])
# backtest_dir = config["user_data_dir"] / "backtest_results"

stats = load_backtest_stats(backtest_dir)
strategy_stats = stats["strategy"][strategy]

df = pd.DataFrame(columns=["dates", "equity"], data=strategy_stats["daily_profit"])
df["equity_daily"] = df["equity"].cumsum()

fig = px.line(df, x="dates", y="equity_daily")
fig.show()


### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [ ]:
from freqtrade.data.btanalysis import load_trades_from_db


# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["exit_reason"].value_counts()


## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with a very high `max_open_trades` setting.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism


# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, "5m")

parallel_trades.plot()


## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import generate_candlestick_graph


# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades["pair"] == pair]

data_red = data["2019-06-01":"2019-06-10"]
# Generate candlestick graph
graph = generate_candlestick_graph(
    pair=pair,
    data=data_red,
    trades=trades_red,
    indicators1=["sma20", "ema50", "ema55"],
    indicators2=["rsi", "macd", "macdsignal", "macdhist"],
)


In [ ]:
# Show graph inline
# graph.show()

# Render graph in a separate window
graph.show(renderer="browser")


## Plot average profit per trade as distribution graph

In [ ]:
import plotly.figure_factory as ff


hist_data = [trades.profit_ratio]
group_labels = ["profit_ratio"]  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size=0.01)
fig.show()


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.